In [9]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import jieba
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from transformers import BertTokenizer, BertForSequenceClassification
import torch
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# 1. 数据加载与预处理
def load_and_clean_data(filepath):
    """加载并清洗评论数据"""
    df = pd.read_csv(netease_comments_167827.csv, encoding='utf-8')
    print(f"原始数据量: {len(df)}条")
    
    # 基础清洗
    df = df.dropna(subset=['content'])  # 删除空评论
    df['content'] = df['content'].apply(lambda x: re.sub(r'[^\w\s]', '', str(x)))  # 去标点
    df['content'] = df['content'].apply(lambda x: re.sub(r'\s+', ' ', x).strip())  # 去空格
    
    # 中文分词
    print("正在进行中文分词...")
    tqdm.pandas(desc="分词进度")
    df['seg_content'] = df['content'].progress_apply(lambda x: ' '.join(jieba.cut(x)))
    
    return df

# 2. 情感分析模型
class SentimentAnalyzer:
    def __init__(self, model_name="bert-base-chinese"):
        """初始化BERT情感分析模型"""
        self.tokenizer = BertTokenizer.from_pretrained(model_name)
        self.model = BertForSequenceClassification.from_pretrained(model_name, num_labels=3)
        self.label_map = {0: "负面", 1: "中性", 2: "正面"}
        
    def predict(self, texts, batch_size=16):
        """批量预测文本情感"""
        results = []
        for i in tqdm(range(0, len(texts), batch_size), desc="情感分析进度"):
            batch = texts[i:i+batch_size]
            inputs = self.tokenizer(
                batch, 
                padding=True, 
                truncation=True, 
                max_length=128, 
                return_tensors="pt"
            )
            with torch.no_grad():
                outputs = self.model(**inputs)
            preds = torch.argmax(outputs.logits, dim=1)
            results.extend(preds.numpy())
        return [self.label_map[p] for p in results]

# 3. 可视化分析
def visualize_results(df):
    """生成分析图表"""
    plt.figure(figsize=(15, 10))
    
    # 情感分布饼图
    plt.subplot(2, 2, 1)
    sentiment_counts = df['sentiment'].value_counts()
    plt.pie(sentiment_counts, labels=sentiment_counts.index, autopct='%1.1f%%')
    plt.title('情感分布比例')
    
    # 词云生成
    plt.subplot(2, 2, 2)
    text = ' '.join(df['seg_content'])
    wordcloud = WordCloud(
        font_path='SimHei.ttf', 
        width=800, 
        height=600, 
        background_color='white'
    ).generate(text)
    plt.imshow(wordcloud)
    plt.axis('off')
    plt.title('高频词云')
    
    # 点赞与情感关系
    plt.subplot(2, 2, 3)
    df.boxplot(column='like_count', by='sentiment', showfliers=False)
    plt.title('不同情感的点赞数分布')
    plt.suptitle('')
    plt.xlabel('情感类型')
    
    # 时间趋势分析（如果有时间字段）
    if 'time' in df.columns:
        plt.subplot(2, 2, 4)
        df['date'] = pd.to_datetime(df['time']).dt.date
        daily_sentiment = df.groupby(['date', 'sentiment']).size().unstack()
        daily_sentiment.plot(kind='line')
        plt.title('情感趋势变化')
    
    plt.tight_layout()
    plt.show()

# 4. 主流程
def main():
    # 加载数据（替换为你的文件路径）
    file_path = "netease_comments_167827.csv"  # 确保包含content字段
    df = load_and_clean_data(file_path)
    
    # 情感分析（使用预训练BERT模型）
    print("\n正在进行情感分析...")
    analyzer = SentimentAnalyzer()
    sample_size = min(1000, len(df))  # 限制分析数量（可选）
    df_sample = df.sample(sample_size, random_state=42) if sample_size < len(df) else df
    df_sample['sentiment'] = analyzer.predict(df_sample['content'].tolist())
    
    # 结果分析
    print("\n情感分类报告:")
    print(classification_report(
        df_sample['sentiment'], 
        df_sample['sentiment'], 
        target_names=['负面', '中性', '正面']
    ))
    
    # 可视化
    visualize_results(df_sample)
    
    # 保存结果
    output_path = "sentiment_analysis_results.csv"
    df_sample.to_csv(output_path, index=False, encoding='utf_8_sig')
    print(f"\n分析结果已保存到: {output_path}")

# 运行主程序
if __name__ == "__main__":
    main()

ImportError: cannot import name 'BertForSequenceClassification' from 'transformers' (E:\Anaconda\Lib\site-packages\transformers\__init__.py)

In [10]:
# 检查每个库是否能正常导入
libs = [
    ('pandas', 'pd'),
    ('numpy', 'np'),
    ('jieba', 'jieba'),
    ('sklearn.model_selection', 'train_test_split'),
    ('sklearn.metrics', 'classification_report'),
    ('matplotlib.pyplot', 'plt'),
    ('wordcloud', 'WordCloud'),
    ('transformers', 'BertTokenizer'),
    ('transformers', 'BertForSequenceClassification'),
    ('torch', 'torch'),
    ('tqdm', 'tqdm')
]

for lib, alias in libs:
    try:
        exec(f"import {lib.split('.')[0]} as {alias}")
        version = eval(f"{alias}.__version__") if hasattr(eval(alias), '__version__') else "版本信息不可用"
        print(f"✅ {lib: <30} 导入成功 | 版本: {version}")
    except ImportError:
        print(f"❌ {lib: <30} 未安装")
    except Exception as e:
        print(f"⚠️ {lib: <30} 导入错误: {str(e)}")

✅ pandas                         导入成功 | 版本: 1.5.3
✅ numpy                          导入成功 | 版本: 1.24.3
✅ jieba                          导入成功 | 版本: 0.42.1
✅ sklearn.model_selection        导入成功 | 版本: 1.3.0
✅ sklearn.metrics                导入成功 | 版本: 1.3.0
✅ matplotlib.pyplot              导入成功 | 版本: 3.7.1
✅ wordcloud                      导入成功 | 版本: 1.9.4
✅ transformers                   导入成功 | 版本: 2.1.1
✅ transformers                   导入成功 | 版本: 2.1.1
❌ torch                          未安装
✅ tqdm                           导入成功 | 版本: 4.65.0


In [ ]:
# 安装必要的库
!pip install snownlp wordcloud jieba

import pandas as pd
import matplotlib.pyplot as plt
from snownlp import SnowNLP
from wordcloud import WordCloud
import jieba
import seaborn as sns
from IPython.display import display, Markdown

# 情感分析函数
def analyze_sentiment(text):
    try:
        s = SnowNLP(text)
        return s.sentiments
    except:
        return 0.5  # 中性作为默认值

# 生成词云函数
def generate_wordcloud(texts, title):
    word_list = []
    for text in texts:
        words = jieba.cut(text)
        word_list.extend([word for word in words if len(word) > 1 and word not in ['哈哈', '啊啊', '呜呜']])
    
    wordcloud = WordCloud(
        font_path='SimHei.ttf',
        background_color='white',
        width=800,
        height=600,
        max_words=100
    ).generate(' '.join(word_list))
    
    plt.figure(figsize=(10, 8))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.title(title, fontsize=15)
    plt.axis('off')
    plt.show()

# 情感分析
display(Markdown("## 🎭 情感分析"))
df['情感分值'] = df['评论内容'].apply(analyze_sentiment)

# 情感分布可视化
plt.figure(figsize=(10, 6))
sns.histplot(df['情感分值'], bins=20, kde=True)
plt.title('评论情感分值分布')
plt.xlabel('情感分值 (0-1, 越接近1表示越积极)')
plt.ylabel('评论数量')
plt.show()

# 情感分类
df['情感分类'] = pd.cut(df['情感分值'], 
                      bins=[0, 0.3, 0.7, 1],
                      labels=['负面', '中性', '正面'])

# 情感分类统计
sentiment_stats = df['情感分类'].value_counts(normalize=True).mul(100).round(1)
display(Markdown("### 情感分类比例 (%)"))
display(sentiment_stats)

# 情感分类饼图
plt.figure(figsize=(8, 8))
plt.pie(sentiment_stats, labels=sentiment_stats.index, autopct='%1.1f%%', startangle=90)
plt.title('评论情感分类比例')
plt.show()

# 点赞数与情感关系
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df, x='情感分值', y='点赞数', alpha=0.6)
plt.title('情感分值与点赞数的关系')
plt.show()

# 用户画像 - 活跃用户
top_users = df['用户'].value_counts().head(5)
display(Markdown("### 🏆 最活跃的5位用户 (评论次数最多)"))
display(top_users)

# 用户画像 - 高影响力用户
top_influencers = df.sort_values('点赞数', ascending=False).drop_duplicates('用户ID').head(5)[['用户', '点赞数', '评论内容']]
display(Markdown("### 💪 最具影响力的5位用户 (单条评论点赞最多)"))
display(top_influencers)

# 词云分析 - 全部评论
display(Markdown("### ☁ 全部评论词云"))
generate_wordcloud(df['评论内容'], '全部评论词云')

# 词云分析 - 正面评论
display(Markdown("### 😊 正面评论词云 (情感分值 > 0.7)"))
positive_comments = df[df['情感分值'] > 0.7]['评论内容']
generate_wordcloud(positive_comments, '正面评论词云')

# 词云分析 - 负面评论
display(Markdown("### 😞 负面评论词云 (情感分值 < 0.3)"))
negative_comments = df[df['情感分值'] < 0.3]['评论内容']
generate_wordcloud(negative_comments, '负面评论词云')

# IP属地分析
if 'IP属地' in df.columns:
    display(Markdown("### 🌍 用户地域分布"))
    location_stats = df['IP属地'].value_counts().head(10)
    plt.figure(figsize=(10, 6))
    location_stats.plot(kind='bar')
    plt.title('评论用户Top10地域分布')
    plt.xlabel('地域')
    plt.ylabel('评论数量')
    plt.xticks(rotation=45)
    plt.show()

# 时间分析
display(Markdown("### ⏰ 评论时间分布"))
df['小时'] = df['评论时间'].dt.hour
hourly_counts = df['小时'].value_counts().sort_index()

plt.figure(figsize=(10, 6))
hourly_counts.plot(kind='bar')
plt.title('24小时评论数量分布')
plt.xlabel('小时')
plt.ylabel('评论数量')
plt.xticks(rotation=0)
plt.show()

# 保存分析结果
analysis_file = f'netease_analysis_{song_id}.csv'
df.to_csv(analysis_file, index=False, encoding='utf-8-sig')
display(Markdown(f"💾 分析结果已保存为 {analysis_file}"))

# 总结报告
display(Markdown("## 📝 分析总结报告"))
display(Markdown(f"""
1. **情感分布**: 歌曲评论中正面情感占比 {sentiment_stats.get('正面', 0)}%，负面情感占比 {sentiment_stats.get('负面', 0)}%
2. **活跃时间**: 评论高峰时段为 {hourly_counts.idxmax()}:00 时，共 {hourly_counts.max()} 条评论
3. **热门地域**: 评论用户主要来自 {location_stats.index[0] if 'IP属地' in df.columns else '未知'}
4. **内容特点**: 高频词汇可通过词云图直观查看，反映了听众的主要关注点
5. **用户互动**: 最高点赞评论获得 {df['点赞数'].max()} 个赞，内容为 "{df.sort_values('点赞数', ascending=False).iloc[0]['评论内容'][:30]}..."
"""))

In [ ]:
# 情感模型优化与智能推荐系统
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from snownlp import SnowNLP
import jieba
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from gensim.models import Word2Vec
from IPython.display import display, Markdown

# 1. 数据准备（接续之前爬取的评论数据）
df = pd.read_csv(f'netease_comments_{song_id}.csv')
print(f"原始数据量：{len(df)}条")

# 2. 情感模型优化（结合领域词典和加权机制）
class EnhancedSentimentAnalyzer:
    def __init__(self):
        # 加载音乐领域情感词典
        self.music_lexicon = {
            '旋律': 0.8, '编曲': 0.7, '作词': 0.6, 
            '青春': 0.9, '回忆': 0.85, '经典': 0.75,
            '难听': -0.9, '过时': -0.6, '失望': -0.7
        }
        self.base_analyzer = SnowNLP
        
    def analyze(self, text):
        try:
            # 基础情感分析
            s = self.base_analyzer(text)
            base_score = s.sentiments
            
            # 领域词典增强
            words = jieba.lcut(text)
            lexicon_score = np.mean([self.music_lexicon.get(word, 0) for word in words])
            
            # 时间加权（夜间评论情感放大）
            hour = pd.to_datetime(df[df['评论内容'] == text]['评论时间'].iloc[0]).hour
            time_weight = 1.2 if 0 <= hour < 6 else 1.0
            
            # 综合计算（基础分60% + 领域分40%）
            final_score = (base_score*0.6 + np.tanh(lexicon_score)*0.4) * time_weight
            return np.clip(final_score, 0, 1)
        except:
            return 0.5

# 应用优化后的情感分析
analyzer = EnhancedSentimentAnalyzer()
df['优化情感分'] = df['评论内容'].progress_apply(analyzer.analyze)

# 对比优化效果
plt.figure(figsize=(10,5))
plt.subplot(121)
df['情感分值'].hist(bins=20)
plt.title('原始情感分布')
plt.subplot(122)
df['优化情感分'].hist(bins=20)
plt.title('优化后情感分布')
plt.show()

# 3. 用户心理分析（LDA主题模型）
def analyze_psychology(texts):
    # 中文分词处理
    chinese_stopwords = set(['的', '了', '和', '是', '我'])
    processed_texts = [
        [word for word in jieba.lcut(text) 
         if len(word) > 1 and word not in chinese_stopwords]
        for text in texts
    ]
    
    # 训练Word2Vec模型
    w2v_model = Word2Vec(processed_texts, vector_size=100, window=5, min_count=2, workers=4)
    
    # TF-IDF向量化
    vectorizer = TfidfVectorizer(max_features=1000)
    tfidf = vectorizer.fit_transform([' '.join(t) for t in processed_texts])
    
    # LDA主题建模
    lda = LatentDirichletAllocation(n_components=3, random_state=42)
    lda.fit(tfidf)
    
    # 可视化主题词
    def show_topics(model, feature_names, n_top_words):
        for topic_idx, topic in enumerate(model.components_):
            display(Markdown(f"**主题#{topic_idx+1}**: " + 
                   ", ".join([feature_names[i] for i in topic.argsort()[:-n_top_words-1:-1]])))
    
    show_topics(lda, vectorizer.get_feature_names_out(), 10)
    
    return lda.transform(tfidf)

# 应用心理分析
topic_probs = analyze_psychology(df['评论内容'])
df['心理主题'] = topic_probs.argmax(axis=1)

# 4. 用户聚类分析
def user_clustering(features):
    # 特征工程（情感分+点赞数+主题概率）
    X = np.column_stack([
        features['优化情感分'],
        np.log1p(features['点赞数']),
        topic_probs
    ])
    
    # 寻找最佳聚类数
    silhouette_scores = []
    for k in range(2, 6):
        kmeans = KMeans(n_clusters=k, random_state=42)
        labels = kmeans.fit_predict(X)
        silhouette_scores.append(silhouette_score(X, labels))
    
    best_k = np.argmax(silhouette_scores) + 2
    display(Markdown(f"### 最佳聚类数：{best_k}（轮廓系数{max(silhouette_scores):.2f}）"))
    
    # 最终聚类
    final_kmeans = KMeans(n_clusters=best_k, random_state=42)
    df['用户类型'] = final_kmeans.fit_predict(X)
    
    # 可视化聚类结果
    plt.figure(figsize=(10,6))
    for cluster in range(best_k):
        cluster_data = df[df['用户类型'] == cluster]
        plt.scatter(
            cluster_data['优化情感分'], 
            np.log1p(cluster_data['点赞数']),
            label=f'类型{cluster}'
        )
    plt.xlabel('优化情感分')
    plt.ylabel('点赞数(log)')
    plt.title('用户聚类分布')
    plt.legend()
    plt.show()
    
    # 分析各类特征
    display(Markdown("### 各类用户平均特征："))
    display(df.groupby('用户类型').agg({
        '优化情感分': 'mean',
        '点赞数': 'median',
        '心理主题': lambda x: x.mode()[0]
    }))

user_clustering(df)

# 5. 智能推荐系统
class MusicRecommender:
    def __init__(self, user_data):
        self.user_profiles = user_data
        self.song_db = {
            167827: {'title': '素颜', 'tags': ['青春', '回忆', '对唱']},
            186016: {'title': '有何不可', 'tags': ['甜蜜', '初恋', '校园']},
            287035: {'title': '雅俗共赏', 'tags': ['哲理', '人生', '思考']},
            5260494: {'title': '乌鸦', 'tags': ['孤独', '成长', '治愈']}
        }
        
    def recommend(self, user_id, top_n=3):
        # 获取用户特征
        user = self.user_profiles[self.user_profiles['用户ID'] == user_id].iloc[0]
        
        # 根据用户类型和心理主题匹配歌曲
        if user['用户类型'] == 0:  # 高情感-怀旧型
            candidates = [sid for sid, meta in self.song_db.items() 
                        if '回忆' in meta['tags']]
        elif user['用户类型'] == 1:  # 理性分析型
            candidates = [sid for sid, meta in self.song_db.items() 
                        if '思考' in meta['tags']]
        else:  # 情感波动型
            candidates = [sid for sid, meta in self.song_db.items() 
                        if '治愈' in meta['tags']]
        
        # 结合时间上下文
        hour = pd.to_datetime(user['评论时间']).hour
        if 0 <= hour < 6:  # 深夜时段推荐舒缓歌曲
            candidates = [sid for sid in candidates 
                         if sid not in [186016]]  # 排除甜蜜歌曲
        
        # 返回推荐结果
        return [(sid, self.song_db[sid]['title']) for sid in candidates[:top_n]]

# 测试推荐系统
recommender = MusicRecommender(df)
sample_user = df.iloc[10]['用户ID']  # 取第10个用户作为示例
rec_results = recommender.recommend(sample_user)

display(Markdown(f"### 为用户ID {sample_user} 生成的推荐："))
for song_id, title in rec_results:
    display(Markdown(f"- {title} (ID: {song_id})"))

# 6. 系统评估
def evaluate_recommendations():
    # 模拟评估（实际应用需A/B测试）
    user_types = df['用户类型'].unique()
    accuracy = {}
    
    for utype in user_types:
        users = df[df['用户类型'] == utype]['用户ID'].sample(5)
        correct = 0
        
        for uid in users:
            recs = [sid for sid, _ in recommender.recommend(uid)]
            user_theme = df[df['用户ID'] == uid]['心理主题'].mode()[0]
            
            # 简单验证：推荐歌曲标签是否匹配用户主题
            matched = any(
                self.song_db[sid]['tags'][0] in ['回忆','青春'] 
                if user_theme == 0 else
                self.song_db[sid]['tags'][0] in ['哲理','思考']
                for sid in recs
            )
            correct += int(matched)
        
        accuracy[utype] = correct / 5
    
    display(Markdown("### 推荐准确率评估（抽样测试）："))
    for utype, acc in accuracy.items():
        display(Markdown(f"- 类型{utype}用户：{acc*100:.1f}%"))

evaluate_recommendations()